#  FAKE NEPALI NEWS DETECTION 

# DATASET 

We have used LIAR Datset which is available on kaggle. The dataset is related to news for fake news detection which consists of six labels, as: 
* half-true 
* false          
* mostly-true    
* true          
* barely-true   
* pants-fire     

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/translated-data-nepali/translated_data_nepali.tsv
/kaggle/input/liar-fake-news-dataset/test.tsv
/kaggle/input/liar-fake-news-dataset/README
/kaggle/input/liar-fake-news-dataset/train.tsv
/kaggle/input/liar-fake-news-dataset/valid.tsv
/kaggle/input/translated-data-english-test/translated_data_english_test.tsv
/kaggle/input/translated-data-nepali-test/translated_data_nepali1_test.tsv
/kaggle/input/translated-data-nepali1/translated_data_nepali1.tsv
/kaggle/input/translated-data-english-validation/translated_data_english_validation.tsv
/kaggle/input/translated-data-english/translated_data_english.tsv
/kaggle/input/translated-data-nepali-validation/translated_data_nepali1_validation.tsv


In [2]:
!pip install inflect
!pip install num2words 
!pip install deep_translator

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 3.5 MB/s eta 0:00:00


**NECESSARY LIBRARY**

In [14]:
import tensorflow as tf
import os
import matplotlib.pyplot as plt 
from deep_translator import GoogleTranslator
import inflect
from num2words import num2words
import string

**READ THE TRAIN,TEST AND VALIDATION DATASET INTO PANDAS DATAFRAME**

In [4]:
df = pd.read_csv('/kaggle/input/liar-fake-news-dataset/train.tsv', sep='\t')

In [5]:
df.head()

,2635.json,false,Says the Annies List political group supports third-trimester abortions on demand.,abortion,dwayne-bohac,State representative,Texas,republican,0,1,0.1,0.2,0.3,a mailer
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [6]:
df.shape

(10239, 14)

In [7]:
for col in df.columns:
    print(col)

2635.json
false
Says the Annies List political group supports third-trimester abortions on demand.
abortion
dwayne-bohac
State representative
Texas
republican
0
1
0.1
0.2
0.3
a mailer


**RENAMING THE COLUMN NAME**
* id (the ID of the statement ([ID].json))
* label 
* statement
* subject
* speaker
* speaker_job_title 
* state_info
* party_affilation
* count_1 (barely true counts)
* count_2 (false counts)
* count_3 (half true counts)
* count_4 (mostly true counts)
* count_5 (pants on fire counts)
* context (venue / location of the speech or statement)

In [8]:
df.columns=['id', 'label', 'statement', 'subjects','speaker', 'speaker_job_title', 'state_info', 'party_affilation',
        # Column 9-13: the total credit history count, including the current statement.
        'count_1', # barely true counts.
        'count_2', # false counts.
        'count_3', # half true counts.
        'count_4', # mostly true counts.
        'count_5', # pants on fire counts.
        
        'context' # Column 14: the context (venue / location of the speech or statement) 
           ]
df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece


In [9]:
df['statement']

0        When did the decline of coal start? It started...
1        Hillary Clinton agrees with John McCain "by vo...
2        Health care reform legislation is likely to ma...
3        The economic turnaround started at the end of ...
4        The Chicago Bears have had more starting quart...
                               ...                        
10234    There are a larger number of shark attacks in ...
10235    Democrats have now become the party of the [At...
10236    Says an alternative to Social Security that op...
10237    On lifting the U.S. Cuban embargo and allowing...
10238    The Department of Veterans Affairs has a manua...
Name: statement, Length: 10239, dtype: object

**Removed the sentence which was unnecessary long because GoogleTranslator provides limit for translation of text above certain character. So while translating, we encoutetred error,so we dropped the two unnecessary rows.**

In [10]:
df['statement'][1279]

'Hospitals, doctors, MRIs, surgeries and so forth are more extensively used and far more expensive in this country than they are in many other countries.\'\'\thealth-care\tmitt-romney\tFormer governor\tMassachusetts\trepublican\t34\t32\t58\t33\t19\ta Fox News Sunday interview\n9874.json\tbarely-true\tObamacare cuts seniors Medicare.\thealth-care,medicare\ted-gillespie\tRepublican strategist\tWashington, D.C.\trepublican\t2\t3\t2\t2\t1\ta campaign email.\n3072.json\tmostly-true\tThe refusal of many federal employees to fly coach costs taxpayers $146 million annually.\tgovernment-efficiency,transparency\tnewsmax\tMagazine and website\tFlorida\tnone\t0\t0\t0\t1\t0\tan e-mail solicitation\n2436.json\tmostly-true\tFlorida spends more than $300 million a year just on children repeating pre-K through 3rd grade.\teducation\talex-sink\t\tFlorida\tdemocrat\t1\t2\t2\t4\t0\tfigures cites on campaign website\n9721.json\ttrue\tMilwaukee County Sheriff David A. Clarke Jr. advised citizens to point th

In [11]:
df.drop(1279, inplace=True)

In [12]:
df.shape

(10238, 14)

# PREPROCESSING

First of all we removed the punctuation from our dataset, and also removed this ("—"), as it was not present in the module of punctuation.

In [13]:
def removepunctuation(sentence):
   new_text = sentence.replace("—", " ") # "—" in the text was not equivalent to "-" so we replcaed "—" with a space
   print 
  #iterate over every single character to see if it is a punctuation or not, and then concatenate them using "join"
   punctuationfree="".join([i for i in new_text if i not in string.punctuation])
   return punctuationfree


**Removed the punctuation from "statement" and added the column as "statement_removedpunctuation"**

In [15]:
df["statement_removedpunctuation"] = df['statement'].apply(lambda x : removepunctuation(x))

In [16]:
df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context,statement_removedpunctuation
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,When did the decline of coal start It started ...
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Hillary Clinton agrees with John McCain by vot...
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,Health care reform legislation is likely to ma...
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,The economic turnaround started at the end of ...
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,The Chicago Bears have had more starting quart...


In [17]:
df['statement_removedpunctuation'][7]

'However it took 195 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Marine Operations CenterPacific'

In [ ]:
# translated = GoogleTranslator(source='auto', target='ne').translate("keep it up, you are awesome")  # output -> Weiter so, du bist großartig
# translated

In [ ]:
# random =  df.sample(10)
# random['statement'].iloc[0]


In [ ]:
# translated = GoogleTranslator(source='auto', target='ne').translate(random['statement'].iloc[0])  # output -> Weiter so, du bist großartig
# translated

In [ ]:
# random['translated_value'] = random['statement'].apply(lambda x: GoogleTranslator(source='auto', target='ne').translate(x))

In [ ]:
# random['translated_value']

In [ ]:
# random['retranslated_value'] = random['translated_value'].apply(lambda x: GoogleTranslator(source='auto', target='en').translate(x))


In [ ]:
# random['retranslated_value'].iloc[0]

In [ ]:
# random['statement'].iloc[2]

In [ ]:
# index=2
# print(random['statement'].iloc[index], "\n", random['translated_value'].iloc[index], "\n",
# random["retranslated_value"].iloc[index])

In [ ]:
# index=3
# print(random['statement'].iloc[index], "\n", random['translated_value'].iloc[index], "\n",
# random["retranslated_value"].iloc[index])

# TRANSLATION FROM ENGLISH "statement" TO NEPALI 

In [19]:
new_df = df.copy()
new_df.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context,statement_removedpunctuation
0,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.,When did the decline of coal start It started ...
1,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver,Hillary Clinton agrees with John McCain by vot...
2,1123.json,false,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release,Health care reform legislation is likely to ma...
3,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN,The economic turnaround started at the end of ...
4,12465.json,true,The Chicago Bears have had more starting quart...,education,robin-vos,Wisconsin Assembly speaker,Wisconsin,republican,0.0,3.0,2.0,5.0,1.0,a an online opinion-piece,The Chicago Bears have had more starting quart...


**"statement_removedpunctuation" is translated from english to nepali and stored into "nepali_translation".**

In [20]:
def translate_to_nepali(text, target_lang):
    # Convert numbers to text using the inflect library
    p = inflect.engine()
    words = text.split()
    for i, word in enumerate(words):
        if word.isnumeric():
            words[i] = p.number_to_words(word)
    text = ' '.join(words)
    
    # Translate the text using GoogleTranslator
    translated_text = GoogleTranslator(source='auto', target=target_lang).translate(text)
    
    return translated_text

In [22]:
new_df['nepali_translation'] = new_df['statement_removedpunctuation'].apply(lambda x: translate_to_nepali(x,"ne"))


In [ ]:
# new_df["nepali_translation"][7]

In [ ]:
# new_df.head()

**SAVED THE  TRANSLATED DATA FRAME**

In [44]:
new_df.to_csv('/kaggle/working/translated_data_nepali1.tsv', sep='\t', index=False)


**LOADED THE TRANSLATED CSV FILE TO THE DATAFRAME**

In [45]:
df1 = pd.read_csv('/kaggle/input/translated-data-nepali-test/translated_data_nepali1_test.tsv', sep='\t')

In [24]:
df1['nepali_translation'][7549]

In [33]:
df1.drop(7549, inplace=True)

In [46]:
df1.shape

(1266, 16)

In [47]:
new_df1 = df1.copy()

In [48]:
new_df1.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context,statement_removedpunctuation,nepali_translation
0,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,Wisconsin is on pace to double the number of l...,विस्कॉन्सिन यस वर्ष छटाउने संख्या दोब्बर गर्न ...
1,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,Says John McCain has done nothing to help the ...,जोन म्याककेनले पशु चिकित्सकहरूलाई मद्दत गर्न क...
2,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,Suzanne Bonamici supports a plan that will cut...,Suzanne Bonamici ले मेडिकेयर एडभान्टेज वरिष्ठह...
3,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,When asked by a reporter whether hes at the ce...,जब एक रिपोर्टरले सोध्यो कि उनी अभियान कानूनको ...
4,5962.json,true,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website,Over the past five years the federal governmen...,विगत पाँच वर्षमा सङ्घीय सरकारले दिवंगत पूर्व स...


**Some of the statements had large number of characters.**

In [27]:
for index, row in df.iterrows():
    statement = row['statement']
    if len(statement) > 1000:
        print(f"Row {index}: Length = {len(statement)}")


Row 6118: Length = 1592
Row 7549: Length = 2099


In [69]:
new_df1['statement'].iloc[921]

'Says Kurt Schrader voted to cut $500 billion from Medicare, threatening thousands of Oregon seniors and their access to their current coverage.'

In [62]:
new_df1.drop(920, inplace=True)

# TRANSLATION FORM NEPALI TO ENGLISH. 

In [70]:
new_df1['english_translation'] = new_df1['nepali_translation'].apply(lambda x:GoogleTranslator(source='auto', target="en").translate(x)
)


translate_to_english(new_df1['nepali_translation'][200],"en")


In [71]:
new_df1.head()

,id,label,statement,subjects,speaker,speaker_job_title,state_info,party_affilation,count_1,count_2,count_3,count_4,count_5,context,statement_removedpunctuation,nepali_translation,english_translation
0,11685.json,false,Wisconsin is on pace to double the number of l...,jobs,katrina-shankland,State representative,Wisconsin,democrat,2,1,0,0,0,a news conference,Wisconsin is on pace to double the number of l...,विस्कॉन्सिन यस वर्ष छटाउने संख्या दोब्बर गर्न ...,Wisconsin is on pace to double the number of l...
1,11096.json,false,Says John McCain has done nothing to help the ...,"military,veterans,voting-record",donald-trump,President-Elect,New York,republican,63,114,51,37,61,comments on ABC's This Week.,Says John McCain has done nothing to help the ...,जोन म्याककेनले पशु चिकित्सकहरूलाई मद्दत गर्न क...,John McCain said nothing was done to help vets
2,5209.json,half-true,Suzanne Bonamici supports a plan that will cut...,"medicare,message-machine-2012,campaign-adverti...",rob-cornilles,consultant,Oregon,republican,1,1,3,1,1,a radio show,Suzanne Bonamici supports a plan that will cut...,Suzanne Bonamici ले मेडिकेयर एडभान्टेज वरिष्ठह...,Suzanne Bonamici supports plan to cut Medicare...
3,9524.json,pants-fire,When asked by a reporter whether hes at the ce...,"campaign-finance,legal-issues,campaign-adverti...",state-democratic-party-wisconsin,NaN,Wisconsin,democrat,5,7,2,2,7,a web video,When asked by a reporter whether hes at the ce...,जब एक रिपोर्टरले सोध्यो कि उनी अभियान कानूनको ...,When a reporter asked if he was at the center ...
4,5962.json,true,Over the past five years the federal governmen...,"federal-budget,pensions,retirement",brendan-doherty,NaN,Rhode Island,republican,1,2,1,1,0,a campaign website,Over the past five years the federal governmen...,विगत पाँच वर्षमा सङ्घीय सरकारले दिवंगत पूर्व स...,"In the last five years, the federal government..."


In [72]:
new_df1.to_csv('/kaggle/working/translated_data_english_test.tsv', sep='\t', index=False)


In [30]:
df2 = pd.read_csv('/kaggle/input/translated-data-english-validation/translated_data_english_validation.tsv', sep='\t')

In [5]:
df2['statement'].iloc[2000]

'70,000 three-year-olds and four-year-olds across America will lose access to the preschool Head Start program. ... 2,000 in the state of Florida alone.'

In [12]:
df2['statement'].iloc[7]

'However, it took $19.5 million in Oregon Lottery funds for the Port of Newport to eventually land the new NOAA Marine Operations Center-Pacific.'

In [6]:
df2['nepali_translation'].iloc[2000]

'फ्लोरिडा राज्यमा मात्रै प्रिस्कूल हेड स्टार्ट कार्यक्रममा अमेरिका भरका ७० हजार तीन वर्षका र चार वर्षका बालबालिकाले पहुँच गुमाउने छन्।'

In [7]:
df2['english_translation'].iloc[2000]

'In the state of Florida alone, 70,000 three- and four-year-old children across the U.S. will lose access to preschool Head Start programs.'

In [9]:
df2['label'].iloc[2000]

'half-true'